必要なライブラリ

In [46]:
import os 
import glob
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torchinfo import summary
import torchvision 
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import joblib

ファイルの読み込み

In [47]:
files = glob.glob('*.csv') 
def getfile(datas, name): 
    return [data for data in datas if name in data][0] 
path_digits_train, path_digits_test, path_digits_sample = getfile(files, 'train'), getfile(files, 'test'), getfile(files, 'sample') 
df_digits_train, df_digits_test, df_digits_sample = pd.read_csv(path_digits_train), pd.read_csv(path_digits_test), pd.read_csv(path_digits_sample) 

欠損値確認

In [48]:
missvalues_train = df_digits_train.isnull().sum()
missvalues_test = df_digits_test.isnull().sum()
missvalues_sample = df_digits_sample.isnull().sum()
print('train:', missvalues_train.value_counts())
print('test:', missvalues_test.value_counts())
print('sample:', missvalues_sample.value_counts())

train: 0    785
Name: count, dtype: int64
test: 0    784
Name: count, dtype: int64
sample: 0    2
Name: count, dtype: int64


データとラベルの分割(データはNumpy型データに変換、その後正規化する)

In [49]:
df_x_train, df_y_train = df_digits_train.drop(columns=['label']), df_digits_train['label'] 
x_train_linear, y_train_linear = df_x_train.to_numpy(), df_y_train.to_numpy() 
x_train_linear = x_train_linear/255 

画像の形状操作、形状確認

In [50]:
x_train_np, y_train_np = x_train_linear.reshape(-1, 28, 28), y_train_linear
print(x_train_np.shape, y_train_np.shape, 'MinMax:', x_train_np.min(), x_train_np.max())

(42000, 28, 28) (42000,) MinMax: 0.0 1.0


Pytorchを使用するための前処理

In [51]:
x_train_tensor = torch.tensor(x_train_np, dtype = torch.float32)
x_train_tensor = x_train_tensor.unsqueeze(1)
y_train_tensor = torch.tensor(y_train_np, dtype = torch.int64)
print(x_train_tensor.shape, y_train_tensor.shape)

torch.Size([42000, 1, 28, 28]) torch.Size([42000])


データの分割

In [52]:
x_train, x_val, y_train, y_val = train_test_split(x_train_tensor, y_train_tensor, test_size = 0.2, random_state = 0)
print(f'x_train:{x_train.shape}, y_train:{y_train.shape}, x_val:{x_val.shape}, y_val:{y_val.shape}')

x_train:torch.Size([33600, 1, 28, 28]), y_train:torch.Size([33600]), x_val:torch.Size([8400, 1, 28, 28]), y_val:torch.Size([8400])


モデルの作成(CNN)

In [53]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1) #(in_C, out_C, kernel_size, stride)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2) #outputsize(h, w, c) = (14, 14, 16)
        
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU(inplace=True)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU(inplace=True)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2) #outputsize(h, w, c) = (7, 7, 32)
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(32*7*7, 256)
        self.relu5 = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(p=0.3)
        self.linear2 = nn.Linear(256, 10)
        
    def forward(self, x):
        out = self.relu1(self.conv1(x))
        out = self.relu2(self.conv2(out))
        out = self.maxpool1(out)
        
        out = self.relu3(self.conv3(out))
        out = self.relu4(self.conv4(out))
        out = self.maxpool2(out)
        
        out = self.avgpool(out)
        out = self.flatten(out)
        out = self.relu5(self.linear1(out))
        out = self.dropout1(out)
        out = self.linear2(out)
        return out 

モデル構造の確認

In [54]:
summary(Net())

Layer (type:depth-idx)                   Param #
Net                                      --
├─Conv2d: 1-1                            160
├─ReLU: 1-2                              --
├─Conv2d: 1-3                            2,320
├─ReLU: 1-4                              --
├─MaxPool2d: 1-5                         --
├─Conv2d: 1-6                            4,640
├─ReLU: 1-7                              --
├─Conv2d: 1-8                            9,248
├─ReLU: 1-9                              --
├─MaxPool2d: 1-10                        --
├─AdaptiveAvgPool2d: 1-11                --
├─Flatten: 1-12                          --
├─Linear: 1-13                           401,664
├─ReLU: 1-14                             --
├─Dropout: 1-15                          --
├─Linear: 1-16                           2,570
Total params: 420,602
Trainable params: 420,602
Non-trainable params: 0

ハイパーパラメータの確定

In [65]:
epochs = 1000
lr = 0.001

def logging_epoch(logs, epoch, loss, accuracy):  
    logs['epoch'].append(epoch) 
    logs['loss'].append(loss) 
    logs['accuracy'].append(accuracy) 

logs = {'train':{'epoch':[], 'loss':[], 'accuracy':[]},
        'val':{'epoch':[], 'loss':[], 'accuracy':[]}} 

net = Net()
optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss() #損失関数の設定：クロスエントロピー誤差

ログの可視化

In [ ]:
def plot_logs(logs):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    #損失関数のグラフ化
    ax[0].plot(logs['train']['epoch'], logs['train']['loss'], label='train', ls='--') #train
    ax[0].plot(logs['val']['epoch'], logs['val']['loss'], label='val') #val
    ax[0].set_xlabel('epoch')
    ax[0].set_ylabel('loss')
    ax[0].set_title('Time series of Loss')
    ax[0].legend(), ax[0].grid()
    #正解率のグラフ化
    ax[1].plot(logs['train']['epoch'], logs['train']['accuracy'], label='train', ls='--') #train
    ax[1].plot(logs['val']['epoch'], logs['val']['accuracy'], label='val') #val
    ax[1].set_xlabel('epoch')
    ax[1].set_ylabel('accuracy')
    ax[1].set_title('Time series of Accuracy')
    ax[1].legend(), ax[1].grid()
    plt.show()

GPUへの転送&バッチ処理

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x_train, x_val, y_train, y_val = x_train.to(device), x_val.to(device), y_train.to(device), y_val.to(device) 

x_train_minibatch = DataLoader(x_train, batch_size=60, shuffle=False)
y_train_minibatch = DataLoader(y_train, batch_size=60, shuffle=False)
x_val_minibatch = DataLoader(x_val, batch_size=60, shuffle=False)
y_val_minibatch = DataLoader(y_val, batch_size=60, shuffle=False)

net= net.to(device) 

バッチ学習

In [ ]:
for epoch in tqdm(range(epochs)):
    train_loss, train_acc  = 0, 0
    val_loss, val_acc  = 0, 0
    
    #訓練フェーズ
    net.train()
    count = 0
    
    for imgs, labels in zip(x_train_minibatch, y_train_minibatch):
        count += len(labels) 
        print(f'epoch: {epoch}, count: {count}, len(labels): {len(labels)}')
        
        #学習フェーズ
        optimizer.zero_grad() #勾配の初期化
        outputs = net(imgs) #順伝播(出力の計算)
        loss = criterion(outputs, labels) #損失関数の計算
        loss.backward()
        optimizer.step() #パラメータ更新
        
        #ロギングデータの更新
        train_loss += loss.item() #損失関数の合計を計算
        y_pred = torch.max(outputs, 1)[1]
        train_acc += (y_pred == labels).sum().item() #正解数を計算
        
        loss_train_avg = train_loss / count #損失関数の平均を計算
        loss_acc_avg = train_acc / count #正解率の平均を計算
    
    #検証フェーズ
    net.eval()
    count = 0
    
    for imgs, labels in zip(x_val_minibatch, y_val_minibatch):
        count += len(labels) #データ数をカウント

        #推論フェーズ
        outputs = net(imgs) #順伝播(出力の計算)
        loss = criterion(outputs, labels) #損失関数の計算

        #ロギングデータの更新
        val_loss += loss.item() #損失関数の合計を計算
        y_pred = torch.max(outputs, 1)[1]
        val_acc += (y_pred == labels).sum().item() #正解数を計算

        loss_val_avg = val_loss / count #損失関数の平均を計算
        loss_acc_avg = val_acc / count #正解率の平均を計算
    
    #学習結果の表示/ロギング
    if epoch % 10 == 0:
        print(f'epoch: {epoch}, loss_train: {loss_train_avg:.4f}, loss_val: {loss_val_avg:.4f}, acc_train: {loss_acc_avg:.4f}, acc_val: {loss_acc_avg:.4f}')
        logging_epoch(logs['train'], epoch=epoch, loss=loss_train_avg, accuracy=loss_acc_avg)
        logging_epoch(logs['val'], epoch=epoch, loss=loss_val_avg, accuracy=loss_acc_avg)
    
plot_logs(logs)

モデルの保存

In [ ]:
torch.save(net.state_dict(), 'models/params_CNN.pth') 
joblib.dump(logs, 'logs_digits_lr0.01_batch.pkl')

学習済みモデルの呼び出し

In [58]:
net = Net()

# モデルのパラメータをロード
model_params = torch.load('model_CNNdigits_lr0.01_batch.pth', map_location=torch.device('cpu')) 
net.load_state_dict(model_params)

<All keys matched successfully>

テスト用データへの前処理

In [64]:
columns = df_digits_sample.columns

x_test_linear = df_digits_test.to_numpy() 
x_test_linear = x_test_linear/255 
x_test_np = x_test_linear.reshape(-1, 28, 28) 
x_test_tensor = torch.tensor(x_test_np, dtype=torch.float32) 
x_test = x_test_tensor.unsqueeze(1) #チャネル数を1にする
print(type(x_test), x_test.shape)

<class 'torch.Tensor'> torch.Size([28000, 1, 28, 28])


予測＆結果のCSVファイル化

In [61]:
outputs = net(x_test) 
predicted_test = torch.max(outputs, 1)[1] 

pred = [i.item() for i in predicted_test]
df = pd.DataFrame([np.arange(1, len(pred)+1), pred]).T
df.columns = columns
display(df.head(3))
df.to_csv('digitsrecog_lr0.01batch.csv', index=False)

,ImageId,Label
0,1,2
1,2,0
2,3,9
